### Restart and Run All

In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 9, 24)

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trade name qty target active reason market xdate'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,ASK,3000,33.50,0,52WL,SET,2022-03-03
1,26,S,ASP,10000,3.82,0,CP1S,SET,2022-08-24
2,3,B,BAM,6000,16.00,0,XXX,SET100,2022-04-29
3,4,B,BANPU,7500,13.00,0,CP1S,SET50,2022-09-14
4,23,S,BCH,3000,20.60,0,C20.50,SET100,2022-08-31
5,5,B,BCP,3000,29.00,0,XXX,SET100,2022-02-02
6,6,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
7,1,B,CPNREIT,10000,18.80,0,HD,SET,2022-08-17
8,24,S,DOHOME,2800,20.30,0,RD20pct,SET100,2022-02-02
9,29,S,GVREIT,10000,9.20,0,DOS,SET,2022-08-26


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate
0,B,ASK,-5,52WL,SET,3000,33.50,34.75,-0.75,-2.11%,0,2022-03-03
1,B,BAM,-11,XXX,SET100,6000,16.00,17.10,-0.30,-1.72%,0,2022-04-29
2,B,BANPU,-4,CP1S,SET50,7500,13.00,13.40,-0.20,-1.47%,0,2022-09-14
3,B,BCP,-10,XXX,SET100,3000,29.00,31.50,-0.25,-0.79%,0,2022-02-02
4,B,BLA,-16,1L,SET50,4000,33.00,37.00,1.00,+2.78%,0,2022-08-23
5,B,CPNREIT,-2,HD,SET,10000,18.80,19.00,0.00,0.00%,0,2022-08-17
6,B,JMART,-9,1L,SET50,3000,48.00,50.25,-1.25,-2.43%,0,2022-04-19
7,B,JMT,3,1L,SET50,1200,73.00,72.25,-2.00,-2.69%,0,2022-04-19
8,B,KSL,-29,6L,SET,30000,3.30,3.88,-0.02,-0.51%,0,2022-02-02
9,B,LANNA,-17,3L,SET,3000,20.80,22.50,-0.60,-2.60%,0,2022-02-02


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0    18
1    12
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

31

In [9]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market,xdate
0,B,ASK,3000,33.50,1,52WL,SET,2022-03-03
1,B,BAM,6000,16.00,0,XXX,SET100,2022-04-29
2,B,BANPU,7500,13.00,1,CP1S,SET50,2022-09-14
3,B,BCP,3000,29.00,0,XXX,SET100,2022-02-02
4,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
5,B,CPNREIT,10000,18.80,1,HD,SET,2022-08-17
6,B,JMART,3000,48.00,1,1L,SET50,2022-04-19
7,B,JMT,1200,73.00,1,1L,SET50,2022-04-19
8,B,KSL,30000,3.30,0,6L,SET,2022-02-02
9,B,LANNA,3000,20.80,0,3L,SET,2022-02-02


In [10]:
rcds = final.values.tolist()
rcds

[['B', 'ASK', 3000, 33.5, 1, '52WL', 'SET', '2022-03-03'],
 ['B', 'BAM', 6000, 16.0, 0, 'XXX', 'SET100', '2022-04-29'],
 ['B', 'BANPU', 7500, 13.0, 1, 'CP1S', 'SET50', '2022-09-14'],
 ['B', 'BCP', 3000, 29.0, 0, 'XXX', 'SET100', '2022-02-02'],
 ['B', 'BLA', 4000, 33.0, 0, '1L', 'SET50', '2022-08-23'],
 ['B', 'CPNREIT', 10000, 18.8, 1, 'HD', 'SET', '2022-08-17'],
 ['B', 'JMART', 3000, 48.0, 1, '1L', 'SET50', '2022-04-19'],
 ['B', 'JMT', 1200, 73.0, 1, '1L', 'SET50', '2022-04-19'],
 ['B', 'KSL', 30000, 3.3, 0, '6L', 'SET', '2022-02-02'],
 ['B', 'LANNA', 3000, 20.8, 0, '3L', 'SET', '2022-02-02'],
 ['B', 'OISHI', 2400, 42.0, 0, '6L', 'SET', '2022-02-02'],
 ['B', 'PSL', 6000, 14.0, 0, '3L', 'SET100', '2022-02-02'],
 ['B', 'PTT', 4500, 35.0, 1, '1L', 'SET50', '2022-09-28'],
 ['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', '2022-02-02'],
 ['B', 'SPALI', 6000, 18.5, 1, '3L', 'SET100', '2022-08-23'],
 ['B', 'SSP', 6000, 8.5, 0, '52WL', 'SET', '2022-02-02'],
 ['B', 'STARK', 25000, 3.9, 1, '3L',

In [11]:
for rcd in rcds:
    print(rcd)

['B', 'ASK', 3000, 33.5, 1, '52WL', 'SET', '2022-03-03']
['B', 'BAM', 6000, 16.0, 0, 'XXX', 'SET100', '2022-04-29']
['B', 'BANPU', 7500, 13.0, 1, 'CP1S', 'SET50', '2022-09-14']
['B', 'BCP', 3000, 29.0, 0, 'XXX', 'SET100', '2022-02-02']
['B', 'BLA', 4000, 33.0, 0, '1L', 'SET50', '2022-08-23']
['B', 'CPNREIT', 10000, 18.8, 1, 'HD', 'SET', '2022-08-17']
['B', 'JMART', 3000, 48.0, 1, '1L', 'SET50', '2022-04-19']
['B', 'JMT', 1200, 73.0, 1, '1L', 'SET50', '2022-04-19']
['B', 'KSL', 30000, 3.3, 0, '6L', 'SET', '2022-02-02']
['B', 'LANNA', 3000, 20.8, 0, '3L', 'SET', '2022-02-02']
['B', 'OISHI', 2400, 42.0, 0, '6L', 'SET', '2022-02-02']
['B', 'PSL', 6000, 14.0, 0, '3L', 'SET100', '2022-02-02']
['B', 'PTT', 4500, 35.0, 1, '1L', 'SET50', '2022-09-28']
['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', '2022-02-02']
['B', 'SPALI', 6000, 18.5, 1, '3L', 'SET100', '2022-08-23']
['B', 'SSP', 6000, 8.5, 0, '52WL', 'SET', '2022-02-02']
['B', 'STARK', 25000, 3.9, 1, '3L', 'SET100', '2022-02-02']
['B', 'T

In [12]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [13]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [14]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)